Install pytorch-metric-lightning and faiss-gpu

In [ ]:
!pip install pytorch-metric-learning
!pip install faiss-gpu

Imports....

In [ ]:
from pytorch_metric_learning import losses, miners, distances, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from torchvision import datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models, datasets, transforms
import numpy as np

from torchvision.datasets import CIFAR10

Create our transform.

Specify our batch size, data loaders, and other miscellanea.

In [ ]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.2435, 0.2616)),
    ])

batch_size = 100

dataset1 = CIFAR10('./', train=True, download=True, transform=transform)
dataset2 = CIFAR10('./', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset1, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=batch_size)

accuracy_calculator = AccuracyCalculator(include = ("precision_at_1",), k = 1)
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)

<h2>Your homework assignment</h2>

Review the code for different losses provided by pytorch-metric-learning: https://github.com/KevinMusgrave/pytorch-metric-learning/tree/master/src/pytorch_metric_learning/losses (click through a few of them to get a hold of how the code looks, the types of parameters you can change, what the different losses are, etc.)

Select three different losses (not including TripletMarginLoss) to train with. Explore the different parameters for those loss functions (e.g., train with different choices).

For each of the different loss functions you are considering, copy the cell below (there should be *four* different cells when you're done -- the one below which provides a "baseline" and then your 3 experiments).



**Changing parameters:**

Parameters that you can change include the embedding size (output_sz), learning rate, margin, the "type_of_triplets" being considered by the miner (you may also explore other miners, although that is not required -- if you do use a different miner, please discuss this in your writeup).

Some caveats for changing parameters:

You may change the embedding size from 64, but you should use the *same* embedding size for your three experiments

Please *do not* change the number of epochs. I recognize that only 2 epochs may prevent your results from converging, but it will give an apples to apples comparison and keep things from taking too long to train.


**Deliverable:**

Provide a write up (~1-2 pages, whatever medium you want) which reports on the achieved accuracy of the different approaches relative to the baseline TripletMarginLoss provided. What parameters did you change and what values did you try? Which achieved the best results? Which of the losses achieved the best accuracy? Where do you think that improvement in accuracy came from?

Upload that write up and your code.

75 points will be assigned based on your write up. 25 points for functional code.

In [ ]:
# you can change these
output_sz = 64 # make sure this is the same across all experiments (you can experiment with different values)
learning_rate = 0.01
margin = 0.2

# don't change this
num_epochs = 2

model = models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_sz) # this "chops off" the pre-trained fc layer, and replaces it with a new random one that's the desired size

model = model.to('cuda') # put this on the gpu

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
distance = distances.CosineSimilarity()

# here are the miner and loss function (the thing you'll be changing to different losses)
miner = miners.TripletMarginMiner(distance = distance, type_of_triplets = "all")
loss_func = losses.TripletMarginLoss(margin = margin, distance = distance)

# train
for epoch in range(1, num_epochs+1):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(data)
        indices_tuple = miner(embeddings, labels)
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print("Epoch {} Iteration {}: Loss = {}, Number of mined triplets = {}".format(epoch, batch_idx, loss, miner.num_triplets))

    train_embeddings, train_labels = get_all_embeddings(dataset1, model)
    test_embeddings, test_labels = get_all_embeddings(dataset2, model)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(test_embeddings, 
                                                train_embeddings,
                                                test_labels,
                                                train_labels,
                                                False)
    print("Epoch: {} | Test set accuracy (Precision@1) = {}".format(epoch, accuracies["precision_at_1"]))